# Libraries

In [1]:
import cv2
from ultralytics import YOLO, solutions
import supervision as sv
import numpy as np
import pandas as pd

# Path of Video

In [2]:
video_path = r'C:\Users\Mohamed Hamde\Data Science\Computer Vision\vehicles-2.mp4'

# Model YOLO

In [3]:
model = YOLO('yolov8n.pt')

# Information about Video

In [110]:
video_info = sv.VideoInfo.from_video_path(video_path)
video_info

VideoInfo(width=1920, height=1080, fps=29, total_frames=1275)

# The Implementation of Counting and Speeding by Supervision

In [255]:
# track object
track = sv.ByteTrack()

# bounding box and label objects
bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annoator = sv.LabelAnnotator()

# line zone
start_point = sv.Point(170, 850)
end_point = sv.Point(1700, 850)
line_zone = sv.LineZone(start_point, end_point)
line_zone_annotator = sv.LineZoneAnnotator(custom_out_text = 'Number of Cars In', 
                                           custom_in_text = 'Number of Cars Out', 
                                           display_in_count=False, display_out_count=False,
                                           text_padding = 10, text_scale=1, text_offset=2, color=sv.draw.color.Color(255,0,0))

# callbacks
def callback(frame: np.ndarray, index:int) -> np.ndarray:
    # detections
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = track.update_with_detections(detections = detections[(detections.class_id == 2) | (detections.class_id == 7)])
    line_zone.trigger(detections)
    
    # as data frame
    ds_counting = {'count_in':[line_zone.out_count], 'count_out':[line_zone.in_count]}
    global df_counting
    df_counting = pd.DataFrame(ds_counting)

    # speed estimation
    prev_center = None

    for box in detections:
        x1, y1, x2, y2 = box[0]
        track_id = box[4]
        center_x = (x1 + x2) / 2 
        center_y = (y1 + y2) / 2  
        center = (center_x, center_y) 

        # Calculate speed if we have a previous center and time
        if prev_center is not None:
            
            displacement = abs(center[0] - prev_center[0])  # Assuming tracking along x-axis
            time_interval = 1 / video_info.fps  # Time between frames
            speed_mps = displacement / time_interval  # Speed in pixels per second (assuming 1 pixel = 1 meter)
            speed_kph = speed_mps * 3.6 # Speed in km/h
            rounded_value  = int(speed_kph)
            first_two_digits = int(float(str(speed_kph)[:2]))
            
            
            x1 = int(x1)
            y1 = int(y1)
            x2 = int(x2)
            y2 = int(y2)            
            
            text = "Speed: 60 km/h"
            font_scale = 0.5
            font_thickness = 1
            font = cv2.FONT_HERSHEY_DUPLEX 
            text_color = (255, 255, 255)  # White text
            bg_color = (0, 255, 0)  # green background color
            
            (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, font_thickness)
            rectangle_color = bg_color
            rectangle_position = ((x1+10, y1+50 - text_height), (x1+10 + text_width, y1+50 + baseline))  
            
            # Draw filled rectangle as background for text
            cv2.rectangle(frame, rectangle_position[0], rectangle_position[1], rectangle_color, cv2.FILLED)
            cv2.putText(frame, f"Speed:{first_two_digits} km/h", (x1+10, y1+50), font, font_scale, (255, 255, 255), font_thickness, cv2.LINE_AA)            
            
        # Update previous center and time
        prev_center = center  
        
    # customization input and output cars 
    font = cv2.FONT_HERSHEY_DUPLEX 
    cv2.putText(frame, f'Number of Cars Out: {line_zone.in_count}', (500, 550),  font, 1,  (255, 255, 0),  3) 
    cv2.putText(frame, f'Number of Cars In: {line_zone.out_count}', (1000, 550),  font, 1,  (255, 255, 0),  3)
    cv2.putText(frame, 'Line Zone', (170, 830),  font, 1.25,  (255, 255, 0),  3)
    
    # annotatios
    labels = [f'{model.model.names[class_id]}' for class_id in detections.class_id]
    annotated_frame = bounding_box_annotator.annotate(scene=frame, detections=detections)
    annotated_frame = label_annoator.annotate(scene=annotated_frame, detections=detections, labels=labels)
    annotated_frame = line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)

    return annotated_frame

# Callback Function

In [257]:
sv.process_video(
    source_path = video_path,
    target_path = 'counting_speeding_objects.mp4',
    callback=callback
)

# Counting and Speeding by Solutions from Ultralytics 

In [193]:
model = YOLO("yolov8n.pt")
names = model.model.names

cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Video writer
video_writer = cv2.VideoWriter("speed_estimation.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

line_pts = [(170, 800), (1700, 800)]

# Init speed-estimation obj
speed_obj = solutions.SpeedEstimator(
    reg_pts=line_pts,
    names=names,
    view_img=True,
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    tracks = model.track(im0, persist=True, show=False)

    im0 = speed_obj.estimate_speed(im0, tracks)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

WARNING  Environment does not support cv2.imshow() or PIL Image.show()
OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


0: 384x640 8 cars, 3 trucks, 189.9ms
Speed: 6.0ms preprocess, 189.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 3 trucks, 183.9ms
Speed: 5.0ms preprocess, 183.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 3 trucks, 183.9ms
Speed: 6.0ms preprocess, 183.9ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 3 trucks, 223.9ms
Speed: 7.0ms preprocess, 223.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 bus, 2 truck

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
